In [2]:
import numpy as np
import math
import random
import time

In [3]:
def create_city (cityfile):
    cityscape = {}
    with open(cityfile, "r") as city_file:
        for line in city_file:
            line = line.split()
            cityscape[line[0]] = (float(line[1]), float(line[2]))
    return cityscape

In [27]:
def distance (i, j):
    x = i[0] - j[0]
    y = i[1] - j[1]
    distance = math.sqrt(x**2 + y**2) #euclidean distance
    return distance

In [30]:
def length_of_tour (path, city):
    length = 0.0
    for i in range(0, len(path)-1):
        length += distance(city[path[i]], city[path[i+1]])
    length += distance(city[path[len(path)-1]], city[path[0]])
    
    return length

In [4]:
def inverse_distance (i, j):
    x = i[0] - j[0]
    y = i[1] - j[1]
    distance = math.sqrt(x**2 + y**2) #euclidean distance
    return 1.0/distance #inverse distance

In [21]:
def init_ph_graph (city):
    
    #obtain a list of nodes
    all_nodes = list(city.keys())
    
    #establish a dictionary of the graph
    ph_graph = {}
    
    for i in range(0, len(all_nodes)):
        for j in range(i, len(all_nodes)): #only do one side, save space
            if i != j: #no self edges
                ph_graph["{0} {1}".format(all_nodes[i], all_nodes[j])] = 0.01 #a small amount > 0
    
    return ph_graph
    

In [39]:
def update_graph (path, ph_graph, city):
    
    #define a list of edges travelled
    all_edges = []
    
    #search all the ant's path
    for i in range(0, len(path)-1):
        all_edges.append((path[i], path[i+1])) #as a tuple
        
    #define ant-k's ph levels
    Lk = length_of_tour(path, city)
    Q = 1.0 #length of NN algorithm
    delt_ph = Q/Lk #equation 3
    p = 0.1 #defined by our problem
    
    #check all edges against ph_graph and set ph, accounting for alphabetical order
    for edge in all_edges:
        
        try:
            ph = (1-p)*ph_graph["{0} {1}".format(edge[0], edge[1])] + delt_ph
            ph_graph["{0} {1}".format(edge[0], edge[1])] = ph #normal order
        except:
            ph = (1-p)*ph_graph["{0} {1}".format(edge[1], edge[0])] + delt_ph
            ph_graph["{0} {1}".format(edge[1], edge[0])] = ph #otherwise try switched
            
    #return updated graph post ant-k's tour
    return ph_graph

In [40]:
a = "a b c d e f g h i j k l m n o p q r".split(" ")
b = "r q p o n m l k j i h g f e d c b a".split(" ")
city = create_city("cities.dat")
ph_g = init_ph_graph(city)
print(length_of_tour(b, city))
print(update_graph(b, ph_g, city))

66.0180207091077
{'a b': 0.024147379295211775, 'a c': 0.01, 'a d': 0.01, 'a e': 0.01, 'a f': 0.01, 'a g': 0.01, 'a h': 0.01, 'a i': 0.01, 'a j': 0.01, 'a k': 0.01, 'a l': 0.01, 'a m': 0.01, 'a n': 0.01, 'a o': 0.01, 'a p': 0.01, 'a q': 0.01, 'a r': 0.01, 'b c': 0.024147379295211775, 'b d': 0.01, 'b e': 0.01, 'b f': 0.01, 'b g': 0.01, 'b h': 0.01, 'b i': 0.01, 'b j': 0.01, 'b k': 0.01, 'b l': 0.01, 'b m': 0.01, 'b n': 0.01, 'b o': 0.01, 'b p': 0.01, 'b q': 0.01, 'b r': 0.01, 'c d': 0.024147379295211775, 'c e': 0.01, 'c f': 0.01, 'c g': 0.01, 'c h': 0.01, 'c i': 0.01, 'c j': 0.01, 'c k': 0.01, 'c l': 0.01, 'c m': 0.01, 'c n': 0.01, 'c o': 0.01, 'c p': 0.01, 'c q': 0.01, 'c r': 0.01, 'd e': 0.024147379295211775, 'd f': 0.01, 'd g': 0.01, 'd h': 0.01, 'd i': 0.01, 'd j': 0.01, 'd k': 0.01, 'd l': 0.01, 'd m': 0.01, 'd n': 0.01, 'd o': 0.01, 'd p': 0.01, 'd q': 0.01, 'd r': 0.01, 'e f': 0.024147379295211775, 'e g': 0.01, 'e h': 0.01, 'e i': 0.01, 'e j': 0.01, 'e k': 0.01, 'e l': 0.01, 'e m'

In [52]:
def do_tour (path, ph_graph, city):
    
    #path is originally just a starting node
    #return should be the input path + one more node travelled (accepted node)
    
    #establish constants
    A = 1
    B = 5
    
    while(True):
        
        #generate all cities not in current path
        rest = []
        for node in city:
            if not node in path:
                rest.append(node)
        
        #exit condition
        if len(rest) == 0:
            return path
                
        current = path[-1] #the last node in the path is the current node i
        potential = rest[random.randint(0,len(rest)-1)] #a random node in the unvisited list is node j
        
        #establish Tij
        try:
            Tij = ph_graph["{0} {1}".format(current, potential)]
        except:
            Tij = ph_graph["{0} {1}".format(potential, current)]
        
        denom = 0.0
        for j in range(0, len(rest)):
            il = rest[j]
            try:
                Til = ph_graph["{0} {1}".format(current, il)]
            except:
                Til = ph_graph["{0} {1}".format(il, current)]
            denom += (Til**A)*(inverse_distance(city[current], city[il])**B)
            
        probability = (Tij**A)*(inverse_distance(city[current], city[potential])**B)/denom
        
        #accept-reject
        if random.random() < probability: #accepted
            path.append(potential) #add to path
    

In [67]:
city = create_city("cities.dat")
print(city.keys())

dict_keys(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r'])


In [73]:
graph = init_ph_graph(city)
ant1 = do_tour(["a"], graph, city)
#print(ant1)
#print(graph)
graph = update_graph(ant1, graph, city)
print(graph)
ant2 = do_tour(["b"], graph, city)
#print(ant2)
#print(graph)
graph = update_graph(ant2, graph, city)
print(graph)
print(ant1, ant2)

{'a b': 0.01, 'a c': 0.03861593800305002, 'a d': 0.01, 'a e': 0.01, 'a f': 0.01, 'a g': 0.01, 'a h': 0.01, 'a i': 0.01, 'a j': 0.01, 'a k': 0.01, 'a l': 0.01, 'a m': 0.01, 'a n': 0.01, 'a o': 0.01, 'a p': 0.01, 'a q': 0.01, 'a r': 0.01, 'b c': 0.01, 'b d': 0.01, 'b e': 0.01, 'b f': 0.01, 'b g': 0.01, 'b h': 0.01, 'b i': 0.01, 'b j': 0.03861593800305002, 'b k': 0.01, 'b l': 0.01, 'b m': 0.01, 'b n': 0.03861593800305002, 'b o': 0.01, 'b p': 0.01, 'b q': 0.01, 'b r': 0.01, 'c d': 0.01, 'c e': 0.01, 'c f': 0.01, 'c g': 0.01, 'c h': 0.01, 'c i': 0.01, 'c j': 0.01, 'c k': 0.01, 'c l': 0.01, 'c m': 0.01, 'c n': 0.03861593800305002, 'c o': 0.01, 'c p': 0.01, 'c q': 0.01, 'c r': 0.01, 'd e': 0.01, 'd f': 0.01, 'd g': 0.01, 'd h': 0.01, 'd i': 0.01, 'd j': 0.01, 'd k': 0.01, 'd l': 0.03861593800305002, 'd m': 0.01, 'd n': 0.01, 'd o': 0.01, 'd p': 0.01, 'd q': 0.03861593800305002, 'd r': 0.01, 'e f': 0.01, 'e g': 0.01, 'e h': 0.01, 'e i': 0.01, 'e j': 0.01, 'e k': 0.03861593800305002, 'e l': 0.0